# Exploring neural networks

## Problem 1

Given a dataset of dimension where each sample contains all 0s and randomly a 1 somewhere to replace the 0s.
Create a neural network that can distinguish between record containing all 0s and those that contain 0s and a 1.